# **The Dealer class**

This notebook provides some documentation about ```Dealer``` objects.

Run the following codeblocks to import the ```Dealer``` class into this notebook.

In [2]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Python\Games\fivecarddraw


In [40]:
from fivecarddraw import Dealer

## **Dealer Functionality**

### **Initialisation**

The ```Dealer``` class creates the following attributes upon initialisation::
* ```Dealer.cards = HandTracker()``` for card based logic.
* ```Dealer.seats = SeatTracker()``` for seat based logic.
* ```Dealer.chips = ChipTracker()``` for chips based logic.
* ```Dealer.action = ActionTracker()``` for player based logic.

More details can be found below. 

### **Providing high-level functionality**

The ```Dealer``` class has the following high-level methods:
* ```Dealer.MoveButton()``` 
* ```Dealer.DealHands()```
* ```Dealer.EditHand(name, discards)```
* ```Dealer.CollectCards()```
* ```Dealer.KickPlayer(name)```
* ```Dealer.TakeAnte()```
* ```Dealer.TakeBet(name, amount)```
* ```Dealer.Payout()```        

### **Setting up game**

The ```Dealer``` class has the following methods for setting up a game:
* ```Dealer.StartingChips(amount)``` 
* ```Dealer.Ante(amount)``` 

### **Providing game info**

The ```Dealer``` class has the following methods for providing game info:
* ```Dealer.PlayerInfo()``` 
* ```Dealer.TableView(name)```
* ```Dealer.Summary()```
